# **<u>3. 다중선형회귀(Multivariable Linear Regression) </u>**
* $x$가 1개인 선형회귀를 단순 선형회귀라고 함. 
* 이번 챕터에서는 다수의 $x$로부터 $y$를 예측하는 **다중선형회귀**에 대해서 학습

## **1.데이터에 대한 이해(Data Definition)**
* 3개의 퀴즈 점수로부터 최종 점수를 예측하는 모델 
  * $x$의 개수가 3개인 훈련데이터
  * $H(x)=w_1x_1+w_2x_2+w_3x_3+b$

## **2.파이토치로 구현하기**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

In [3]:
torch.manual_seed(1) #랜덤 시드 고정

In [4]:
# 훈련데이터 선언
x1_train=torch.FloatTensor([[73],[93],[89],[96],[73]])
x2_train=torch.FloatTensor([[80],[88],[91],[98],[66]])
x3_train=torch.FloatTensor([[75],[93],[90],[100],[70]])

y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

In [5]:
# 가중치 w와 편향 b 선언
w1=torch.zeros(1,requires_grad=True)
w2=torch.zeros(1,requires_grad=True)
w3=torch.zeros(1,requires_grad=True)

b=torch.zeros(1,requires_grad=True)

In [6]:
#optimizer 설정
optimizer=optim.SGD([w1,w2,w3,b],lr=1e-5)
nb_epochs=1000


for epoch in range(nb_epochs+1):
    #H(x)
    hypothesis=x1_train*w1+x2_train*w2+x3_train*w3+b
    
    #cost
    cost=torch.mean((y_train-hypothesis)**2) 

    #cost로 H(x) 개선
    optimizer.zero_grad() # 파라메터의 gradient 0으로 초기화
    cost.backward() #gradient 계산
    optimizer.step() #update parameter 

    #100번마다 로그 출력
    if epoch % 100==0:
        print('Epoch {:4d}/{} w1:{:.3f} w2:{:.3f} w3:{:.3f} b{:.3f} Cost:{:.6f}'.format(epoch,nb_epochs,w1.item(),w2.item(),w3.item(),b.item(),cost.item()))

Epoch    0/1000 w1:0.294 w2:0.294 w3:0.297 b0.003 Cost:29661.800781
Epoch  100/1000 w1:0.674 w2:0.661 w3:0.676 b0.008 Cost:1.563634
Epoch  200/1000 w1:0.679 w2:0.655 w3:0.677 b0.008 Cost:1.497607
Epoch  300/1000 w1:0.684 w2:0.649 w3:0.677 b0.008 Cost:1.435026
Epoch  400/1000 w1:0.689 w2:0.643 w3:0.678 b0.008 Cost:1.375730
Epoch  500/1000 w1:0.694 w2:0.638 w3:0.678 b0.009 Cost:1.319511
Epoch  600/1000 w1:0.699 w2:0.633 w3:0.679 b0.009 Cost:1.266222
Epoch  700/1000 w1:0.704 w2:0.627 w3:0.679 b0.009 Cost:1.215696
Epoch  800/1000 w1:0.709 w2:0.622 w3:0.679 b0.009 Cost:1.167818
Epoch  900/1000 w1:0.713 w2:0.617 w3:0.680 b0.009 Cost:1.122429
Epoch 1000/1000 w1:0.718 w2:0.613 w3:0.680 b0.009 Cost:1.079378


## **3. 벡터와 행렬 연산으로 바꾸기**
* $x$의 개수가 1000개라고 가정하면 x_train1~ x_train1000, w1~w1000까지 총 2000개의 변수를 선언해야함. 
* $x$와 $w$ 변수 선언을 효율적으로 처리하려면 어떻게 해야하는가. 
* **행렬 곱셈 연산(또는 벡터의 내적)** 사용 
### **(1) 벡터연산으로 이해하기**
$$ \big(\begin{matrix}x_1&x_2&x_3 \end{matrix}\big)  . \big(\begin{matrix}w_1\\w_2\\ w_3\\ \end{matrix}\big) = (x_1w_1+x_2w_2+x_3w_3)$$
* 두 벡터를 각각 $X$와 $W$로 표현한다면, hypothesis는 다음과 같이 표현됨. 
$H(X)=XW$
* $x$의 개수가 3개였음에도, $X$와 $W$라는 두 개의 변수로 표현됨. 

### **(2) 행렬 연산으로 이해하기**
$$ \big(\begin{matrix}x_{11}&x_{12}& x_{13}\\x_{21}&x_{22}& x_{23}\\x_{31}&x_{32}& x_{33}\\x_{41}&x_{42}& x_{43}\\x_{51}&x_{52}& x_{53} \end{matrix}\big)  . \big(\begin{matrix}w_1\\w_2\\ w_3\\ \end{matrix}\big) = \big(\begin{matrix}x_{11}w1+x_{12}w2+x_{13}w3\\x_{21}w1+x_{22}w2+x_{23}w3\\x_{31}w1+x_{32}w2+x_{33}w3 \\x_{41}w1+x_{42}w2+x_{43}w3\\x_{51}w1+x_{52}w2+x_{53}w3\end{matrix}\big)$$
* $x$의 개수가 5개인 경우 가중치와의 곱은 다음과 같이 표현할 수 있음. 
* 만약 여기에 **편향 b를 추가**한다면, 
$$ \big(\begin{matrix}x_{11}&x_{12}& x_{13}\\x_{21}&x_{22}& x_{23}\\x_{31}&x_{32}& x_{33}\\x_{41}&x_{42}& x_{43}\\x_{51}&x_{52}& x_{53} \end{matrix}\big)  . \big(\begin{matrix}w_1\\w_2\\ w_3\\ \end{matrix}\big) +\big(\begin{matrix}b\\b\\b\\b\\b\\ \end{matrix}\big)= \big(\begin{matrix}x_{11}w1+x_{12}w2+x_{13}w3+b\\x_{21}w1+x_{22}w2+x_{23}w3+b\\x_{31}w1+x_{32}w2+x_{33}w3 +b\\x_{41}w1+x_{42}w2+x_{43}w3+b\\x_{51}w1+x_{52}w2+x_{53}w3+b\end{matrix}\big) $$
* 이는 결과적으로 다음과 같음. 
$H(X)=XW+B$

## **4. 행렬 연산을 고려하여 파이토치로 구현하기**
* 훈련 데이터는 행렬의 형태를 가지므로, 훈련데이터 또한 행렬로 선언해야함.

In [9]:
x_train=torch.FloatTensor([[73, 80, 75],
                           [93, 88, 93],
                           [89,91,80],
                           [96,98,100],
                           [73,66,70]])
y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [14]:
# 가중치와 편향 선언 (모델 초기화)
W=torch.zeros((3,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

#optimizer 설정
optimizer=optim.SGD([W,b],lr=1e-5) 

In [15]:
nb_epochs=20

for epoch in range(nb_epochs+1):

    #H(x) 계산
    #편향 b는 브로드캐스팅되어 각 샘플에 더해짐
    hypothesis=x_train.matmul(W)+b

    #cost 계산 
    cost=torch.mean((y_train-hypothesis)**2)

    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis{} Cost:{:.6f}'.format(epoch,nb_epochs,hypothesis.squeeze().detach(),cost.item()))

Epoch    0/20 hypothesistensor([0., 0., 0., 0., 0.]) Cost:29661.800781
Epoch    1/20 hypothesistensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) Cost:9537.694336
Epoch    2/20 hypothesistensor([104.5421, 125.6208, 119.2478, 134.7862,  95.8280]) Cost:3069.590088
Epoch    3/20 hypothesistensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) Cost:990.670898
Epoch    4/20 hypothesistensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) Cost:322.482086
Epoch    5/20 hypothesistensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) Cost:107.717064
Epoch    6/20 hypothesistensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]) Cost:38.687496
Epoch    7/20 hypothesistensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]) Cost:16.499043
Epoch    8/20 hypothesistensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]) Cost:9.365656
Epoch    9/20 hypothesistensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625]) Cost:7.071114
Epoch   10/20 hypothesistensor([153.5285, 184.4